# Exploración de los datos
## Proyecto de Aprendizaje Automático
#### Pablo Berástegui Magallón (202311460)
#### Ingeniería Matemática e Inteligencia Artificial
#### Universidad Pontificia Comillas

____________________________
### Importación de las librerías necesarias

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils import optimal_bins, categories_map, variable_name_map

____________________________
### Inicialización del dataset

In [98]:
df = pd.read_csv('../data/rendimiento_estudiantes_test_vacio.csv', sep=',')
df

,escuela,sexo,edad,entorno,TamFam,EstPadres,Medu,Pedu,Mtrab,Ptrab,...,TiempoLib,SalAm,AlcSem,AlcFin,salud,faltas,asignatura,T1,T2,T3
0,BG,F,17,U,>=4,J,1,1,casa,servicios,...,3,3,1,1,3,0,M,8,8,NaN
1,BG,M,16,R,>=4,J,4,4,docencia,docencia,...,5,5,2,5,4,8,L,14,14,NaN
2,BG,M,17,U,<4,J,2,3,servicios,servicios,...,3,3,1,3,3,0,L,10,11,NaN
3,BG,F,15,U,>=4,J,2,3,otros,otros,...,5,1,1,1,5,4,L,11,11,NaN
4,BG,M,15,R,>=4,J,4,3,docencia,casa,...,5,2,1,1,5,0,L,13,14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,BG,F,16,U,<4,J,2,2,otros,otros,...,3,3,1,2,3,25,M,7,10,NaN
205,BG,F,15,U,>=4,J,1,2,casa,otros,...,3,2,1,1,5,6,L,13,12,NaN
206,BG,M,17,U,<4,S,3,2,docencia,servicios,...,4,4,3,4,3,19,M,11,9,NaN
207,BG,F,15,U,>=4,S,3,3,otros,sanidad,...,3,3,1,1,4,10,L,10,10,NaN


____________________________
### Exploración inicial

In [99]:
n = df.shape[0]
print(f'Número de muestras (n): {n}')

p = df.shape[1] - 1
print(f'Número de variables (p): {p}')

null_lines = df[df.isna().any(axis=1)].shape[0]
print('Muestras con algún dato NaN: ', null_lines)

Número de muestras (n): 209
Número de variables (p): 33
Muestras con algún dato NaN:  209


In [100]:
for col in df.columns:
    if df[col].isna().any():
        print(f'Variable {col} tiene {df[col].isna().sum()} valores NaN')
        print(f'Porcentaje de NaN: {df[col].isna().sum() / n * 100:.2f}%')
        print('-----------------------------------')

Variable T3 tiene 209 valores NaN
Porcentaje de NaN: 100.00%
-----------------------------------


In [101]:
df['razon'] = df['razon'].apply(lambda x: 'otros' if x == 'otras' else x)
df['razon'].unique()

array(['optativas', 'reputacion', 'otros', 'cercania'], dtype=object)

In [102]:
outliers = df['faltas'][df['faltas'] > 200]

Por el momento decidimos eliminar dichos valores por comodidad.

In [103]:
df['faltas'] = df['faltas'].drop(outliers.index)
df['faltas'] = df['faltas'].fillna(0).astype(int)

En nuestro dataset hay tres bloques importantes de correlación:
- `T1`, `T2` y `T3`
- `Pedu`, `Medu`
- `AlcSem`, `AlcFin`

Vamos a simplificar los dos últimos bloques mencionados para reducir la dimensión.

Consideremos que el nivel "promedio" de educación de los padres es el que influye al niño en sus resultados académicos.

In [104]:
PMEdu = (df["Medu"] + df["Pedu"]) / 2
df.insert(df.columns.get_loc("Pedu") + 1, "PMedu", PMEdu)
df.drop(columns=['Pedu', 'Medu'], inplace=True)

Por otro lado, consideramos que es más influyente la cantidad de alcohol consumida entre semana que en fines de semana, por eso hacemos dicha ponderación.

In [ ]:
Alc = df['AlcSem'] * 0.7 + df['AlcFin'] * 0.3
Alc = Alc.round(0).astype(int)
df.insert(df.columns.get_loc("AlcSem") + 1, "Alc", Alc)
df.drop(columns=['AlcSem', 'AlcFin'], inplace=True)

In [106]:
df.to_csv('../data/rendimiento_estudiantes_test_vacio_clean.csv')